<a href="https://colab.research.google.com/github/huangjihui511/seq2seq-dialog/blob/master/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  免费用 Google GPU 来 finetune GPT-2

英文来自： [Max Woolf](http://minimaxir.com)

中文修改：Andy

最近 GPT2 放出了 345M 的中型模型（之前是 117M 模型），于是又引起一波 Finetune 大潮，于是自己也就顺便抽出时间拿这个 345M 模型 finetune 了几个玩。

这部分主要是教大家用 `gpt-2-simple` 库，并且蹭 Google Colab 免费 GPU 来对 GPT2 模型进行 finetune，这篇 Notebook 业主要来自这个库的作者。


 `gpt-2-simple` 库 github 地址：https://github.com/minimaxir/gpt-2-simple



In [1]:
!pip install tensorflow==1.15.2


     |████████████████████████████████| 110.5MB 88kB/s 
     |████████████████████████████████| 512kB 42.9MB/s 
     |████████████████████████████████| 3.8MB 37.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ee48470b65a32b8a53260149ef6b0f380464018f525a6976b0f1d3ec50af5420
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

In [1]:
import tensorflow.contrib.graph_editor as ge

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



#### 开始前准备

1. 首先，要想薅 Google 羊毛，就得现有个 Google 账号，[注册](https://accounts.google.com/signup);
2. 然后把这个 Notebook 保存到自己 Google Drive 里去，便于运行和修改。(File -> Save a Copy in Drive)
3. 之后，同样的既然要薅 Google 羊毛，就还得用他家产品，所以保证用 Chrome 浏览器；
4. 最后，安装环境和 import 各种需要包，运行下面的 Cell 就好了:

In [0]:
!pip install -q gpt_2_simple

import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## 查看 GPU

既然要薅羊毛，那就要薅个清楚，先来看看 GPU 信息吧。

现在 Colab 更新过一次，已经再使用英伟达 T4 GPU了，比之前的 K80 要快些，而且内存也要大些，有 16 G。这么大的内存 finetune 的时候拿大模型来也都没问题了，而且可以生成更多文本。


In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## 下载 GPT-2 模型

首先，finetune 的第一步，先把预训练模型下载下来，就像做菜要先备好材料。

正如开头提到的，现在放出了两个 GPT-2 模型， 
* `117M` : 小模型，大概有 500MB 大小；
* `345M`: 最近放出的中型模型，1.5 GB.

之后 finetune 的时候，只需要修改模型名字 `model_name` 参数就可以选择模型。大模型有更多的知识，但同时也要更长时间来进行 finetune 和生成。

下面的 cell 就是用来下载预训练模型文件的，会保持在 Colab 的 `/models/<model_name>`下，点左边栏上面的 `Files` 也可以看到。

模型不是永久保存的，下次 finetune 的时候还要重新下载。

In [4]:
gpt2.download_gpt2(model_name="774M")

Fetching checkpoint: 1.05Mit [00:00, 246Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 90.3Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 684Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:44, 70.2Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 266Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 172Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 158Mit/s]                                                       


## 装载 Google Drive （类似云盘）

想输入训练数据，然后保存模型的最好方法，就是把 Google Drive 给装载进来，之后把训练数据放在上面，保存也直接放上面


下面这个 cell 就是装载个人 Google Driver 的命令。会出现一个链接和提示框，点击链接获得授权码，粘贴到提示框里去。

In [5]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 上传训练文本

也是在左边的侧栏里面，直接点击`Files` 里面的上传 `UPLOAD` 就好了。

![alt text](https://i.imgur.com/TGcZT4h.png)

之后也可以传任意自己想 finetune 的文本，这里我们用老友记十季的剧本来作为训练数据。直接上传的话推荐小于 10M 的数据。


In [0]:
file_name = "COVID-GPT2.txt"

如果大于 10M 的文本，那就推荐先上传到 google drive (根目录) 里然后再拷过来。如果报错，再重新运行装载命令。

In [0]:
# gpt2.copy_file_to_gdrive(file_name)
gpt2.copy_file_from_gdrive(file_name)


## Finetune GPT-2

于是准备好模型和数据，就要开始今天的主菜了，finetune GPT2. 

下面的代码会运行一个 Tensorflow Sessioin，读取训练设定，然后按照设定训练。`steps` 用来指定 finetune 多少步，如果设成 -1 的话就表示不停 finetune 下去。

模型文件会保存在`/checkpoint/run1` 里面，会根据`save_every`来保存一定步数文件，结束训练的时候也会保存。

训练结束需要大概半个小时，一定要记得保存模型。



**重要提示**：如果想要重复运行下面 cell 的话，需要重启服务，`Runtime/Restart Runtime`.

gpt2.finetune 训练参数介绍：


*  **`restore_from`**:  `fresh` 是指从 GPT2 原模型开始, 而 `latest`是从之前 finetune 保存的模型继续训练.
* **`sample_every`**: 每多少步输出样本，看看训练效果
* **`print_every`**: 每多少步打印训练的一些参数，从左到右，步数、时间，loss，平均loss
* **`learning_rate`**: 学习率 (默认 `1e-4`, 如果数据小于1MB的话可以调低到 `1e-5`)
*  **`run_name`**: 运行的时候，保存模型到`checkpoint`下哪个子文件，默认 `run1`

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='774M',
              steps=200,
              restore_from='latest',
              print_every=1,
              sample_every=10,
              save_every=50
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


模型训练完之后，你就可以把保存模型，直接拷到 Google Drive 下面去了。


之后如果想在自己本地使用的话，就直接从 Google Drive 直接下载就可以了。

In [0]:
gpt2.copy_checkpoint_to_gdrive()

OK，搞定了。

现在就尽情调戏自己训练好的模型吧。

## 载入训练好的模型

下面的 cell 会把你 google drive 下的 `checkpoint` 拷进来。

如果是一口气直接训练过来的话，就不用运行这一步了。

In [0]:
gpt2.copy_checkpoint_from_gdrive()

之后就是，从模型中载入训练好的模型。

这个 cell 和之前训练 cell 一样，如果想重复跑的话得重启环境。


In [7]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

Loading checkpoint checkpoint/run1/model-200
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-200


## 从训练好模型生成文本

载入模型之后，就是激动人心的生成部分了！

下面这个是无条件（Unconditional）的生成，也就是没有任何限制，让模型自己生成。

结果并不理想，可能因为 finetune 的步数太小了，还有一些生成参数没有设置，这里只是个示例所以就只设 1000 步训练，想要更好效果可以设更大一些。

In [0]:
gpt2.generate(sess)

之后如果你想要创建 API 获得生成的文本的话，直接运行 `text = gpt2.generate(sess, return_as_list=True)[0]` 就好了。

然后，上面提到了无条件生成，那有条件生成(Conditional)呢，我们可以直接用`prefix`来指定一段文本，然后让模型来续写。

此外，还可以同时生成几个样本，通过指定`nsamples`。还有，通过设定`batch_size`还可以加速生成过程。


`gpt2.generate` 和相关函数其他一些有用的选项：

*  **`length`**: 生成文本长度 (默认 1023, 也是可设最大长度)
* **`temperature`**: temperature 越高，生成的就随意。 (默认 0.7，推荐 0.7 到 1.0之间)
* **`top_k`**: 将输出限定在 top k 里面 (默认0，也就是不使用。推荐在生成效果差的时候使用，可以设top_k=40)
* **`truncate`**: 从指定符号阶段生成文本 (比如设 `truncate='<|endoftext|>'`, 那么就会取第一个'<|endoftext|>'前的文本作为输出). 可以和一个比较小的`length`值搭配使用.
*  **`include_prefix`**: 如果用了 `truncate` 和 `include_prefix=False`, 那么在返回文本中就不会包含`prefix`里的文本。

In [9]:
gpt2.generate(sess,
              length=100,
              temperature=0.7,
              prefix="Patient: Hello doctor, I get a cough for the last few days, which is heavy during night times .  No raise in temperature but feeling tired with no travel history .  No contact with any Covid-19 persons .  It has been four to five days and has drunk a lot of Benadryl and took Paracetamol too .  Doctors have shut the OP so do not know what to do? Please help . ",
              nsamples=20,
              batch_size=5,
              top_k=40
              )

Patient: Hello doctor, I get a cough for the last few days, which is heavy during night times .  No raise in temperature but feeling tired with no travel history .  No contact with any Covid-19 persons .  It has been four to five days and has drunk a lot of Benadryl and took Paracetamol too .  Doctors have shut the OP so do not know what to do? Please help .  Thanks
Doctor: Hello,CT scan is diagnostic for COVID-19 and if cough is severe enough, it can progress into pneumonia .  So better to consult pulmonologist and get done clinical examination of respiratory system and PFT (Pulmonary Function Test) .  PFT will not only diagnose pneumonia but it will also tell you about severity of the cough .  You may need antibiotics and other treatment . Hope I have answered to your query .  Let me know if I can assist
Patient: Hello doctor, I get a cough for the last few days, which is heavy during night times .  No raise in temperature but feeling tired with no travel history .  No contact with a

如果想要生成大量文本的话，就可以用下面 cell 的命令。

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

files.download(gen_file)

# 额外提示

如果出现错误 (比如 GPU Sync Fail or out-of-memory/OOM), 可以用下列命令来强行杀掉并重启:

In [0]:
!kill -9 -1

欢迎转载，务必注明出处。


关注公众号：Andy的写作间(andy_writing) ，更多自然语言处理，深度学习，AI，写作等等等